# 📓 The GenAI Revolution Cookbook

**Title:** How to Set an OpenAI Data Retention Policy for Prompts, Logs

**Description:** Achieve GDPR-ready GenAI retention: classify prompts, outputs, and logs, configure OpenAI/Azure settings, and automate lifecycle deletion with audits and reporting.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



## Why retention policy matters for GenAI

GenAI systems process sensitive prompts, outputs, and logs that trigger GDPR storage limitation, HIPAA retention rules, and sector-specific mandates. Without a clear retention policy, you face regulatory fines, audit failures, and runaway storage costs. This guide shows you how to stand up a GDPR-ready retention policy in 30 days with zero data retention (ZDR) configured, deletion automation live, and audit evidence ready—targeting under 2% deletion failures per week and no objects older than 90 days. You'll learn which decisions to make, who owns each step, and what artifacts to produce so your team can execute confidently.

**Who you need:** Engage your Legal/DPO, Security, Data Platform, and App Owners from day 1. This is a cross-functional effort, and early alignment prevents rework.

**Scope:** This guidance targets OpenAI API and Azure OpenAI deployments with Azure as the primary automation surface. If you use OpenAI directly or run on AWS/GCP, adapt the automation steps to your platform's serverless and storage lifecycle tools. Out of scope: consumer ChatGPT UI, non-Azure cloud equivalents, and deep multi-cloud orchestration.

## Classify GenAI data and map retention requirements

Start by classifying data into four buckets: prompts (inputs users send), outputs (model responses), interaction logs (threads, errors, telemetry), and uploaded files/fine-tuning datasets. Each class carries different sensitivity and utility, so you'll set distinct retention windows and controls for each to avoid blanket policies that over-retain. For practical advice on building governance structures that support both innovation and compliance, see our guide on [managing GenAI tooling adoption for technical teams](/article/ai-powered-tools-for-software-development-how-to-lead-adoption-2).

**Leader Decision Checklist:**
- **What to decide:** Retention window per data class (e.g., prompts 30 days, outputs 60 days, logs 90 days, training data 1 year or until project end).
- **Who owns it:** Legal/DPO sets minimum retention; business units propose extensions with justification; Security reviews for incident response needs.
- **Evidence to collect:** Documented retention register mapping each data class to retention period, lawful basis, and owner.
- **Success metric:** 100% of data classes have approved retention windows and documented lawful basis before automation begins.

Map your retention drivers: GDPR Article 5(1)(e) mandates storage limitation; HIPAA requires 6 years for certain records; financial services may demand 7+ years for audit trails. Document conflicts (e.g., legal recordkeeping vs. storage limitation) and resolve them with your DPO—record exceptions and the rationale in your retention register.

**Privacy governance integration:**
- Run a Data Protection Impact Assessment (DPIA) if your GenAI processing involves high-risk personal data or large-scale profiling.
- Update your Record of Processing Activities (RoPA) entry to reflect GenAI data flows, retention periods, and lawful basis per use case.
- Execute Data Processing Agreements (DPAs) and Standard Contractual Clauses (SCCs) with OpenAI/Azure; store signed copies in your evidence pack and reference them in the retention register.

**Discovery and centralization:**
- Conduct a one-time discovery sweep using Microsoft Purview or equivalent to inventory shadow GenAI data across subscriptions, storage accounts, and business units.
- Migrate discovered data into centralized, governed storage (e.g., dedicated Azure Storage accounts with lifecycle policies) before applying retention rules.
- Define region-specific retention and segregation for multi-region or multi-business-unit deployments; document cross-border transfer approvals and data residency requirements.

**Data Subject Request (DSR) workflow:**
- Define SLA owners and searchable indexes/metadata to fulfill access and erasure requests across prompts, outputs, logs, vector stores, and vendor systems.
- Document query patterns (e.g., user ID, session ID) and deletion propagation paths to ensure complete erasure.
- Specify how legal holds interact with DSRs—legal hold data must be preserved even if a DSR is received; document the hold release process.

**Backup and restore:**
- Set explicit backup retention windows (e.g., 30-day rolling backups for operational recovery) and purge-on-restore procedures to avoid reintroducing deleted data.
- For immutable backups, document GDPR erasure handling (e.g., metadata marking, access restrictions) and who approves exceptions.

**Pilot-first approach:**
- Start with a single use case and business unit to validate your retention policy, automation, and audit evidence before enterprise rollout.
- Define go/no-go criteria: deletion error rate below 2%, audit log completeness at 100%, vendor baseline captured, and Legal/DPO sign-off.

**Evidence to collect:** Retention register schema, DPIA summary, RoPA entry, signed DPAs/SCCs, discovery inventory, DSR workflow diagram, backup retention policy, pilot go/no-go checklist.

## Configure vendor settings and centralize sensitive artifacts

Enable Azure OpenAI zero data retention (ZDR) so Microsoft does not store prompts or outputs for abuse monitoring or model improvement. In the Azure portal, navigate to your Azure OpenAI resource, select Data Management, and toggle on "Opt out of logging for abuse monitoring." Confirm the setting is applied across all deployments.

**ZDR trade-offs and compensating controls:**
- ZDR reduces abuse and fraud signal coverage. Implement application-side abuse detection (e.g., rate limits, prompt filtering, anomaly detection) to compensate.
- Some Azure OpenAI features may require logging; document feature eligibility and who signs off on the ZDR trade-off.

**How we execute:** Azure OpenAI ZDR is a tenant-level setting; verify it in the Azure portal under your resource's Data Management blade. For OpenAI API (non-Azure), review your organization's data retention settings at platform.openai.com and disable data retention for model training.

Centralize sensitive artifacts—fine-tuning datasets, vector embeddings, uploaded files—in dedicated Azure Storage accounts with private endpoints and role-based access control (RBAC). Tag each container with metadata (data class, retention period, business owner) so lifecycle policies can target them precisely. Avoid scattering data across subscriptions or untagged storage, which makes compliance audits painful.

**Leader Decision Checklist:**
- **What to decide:** ZDR on/off, compensating controls for abuse detection, storage account topology (single vs. multi-region, per-business-unit segregation).
- **Who owns it:** Security approves ZDR trade-offs and compensating controls; Data Platform provisions storage and RBAC; App Owners tag containers.
- **Evidence to collect:** Screenshots of ZDR toggle, storage account topology diagram, RBAC assignments, container tagging schema.
- **Success metric:** ZDR enabled and verified; 100% of sensitive artifacts in governed storage with correct tags before lifecycle rules deploy.

**Vendor contracting and audit:**
- Store signed DPAs, SCCs, and security annexes in a central repository (e.g., SharePoint, contract management system).
- Link each vendor agreement to the relevant data class in your retention register for audit traceability.

**Multi-region and business unit segmentation:**
- For enterprises with mixed jurisdictions, define region-specific retention (e.g., EU data in EU storage with GDPR retention, US data with state-specific rules).
- Use separate storage accounts or tenants per region/business unit to enforce segregation; document cross-border transfer approvals.

**Evidence to collect:** ZDR confirmation screenshots, storage account topology diagram, RBAC role assignments, container tagging schema, vendor agreement repository index, region-specific retention matrix, Security sign-off on ZDR and compensating controls.

## Automate deletion with lifecycle policies and serverless jobs

Azure Storage lifecycle management lets you define rules that transition or delete blobs based on age or last-modified date. In the Azure portal, go to your storage account, select Lifecycle Management, and add a rule: "Delete blobs in container 'prompts' if last modified >30 days ago." Repeat for each data class with its approved retention window. Test rules in a non-production account first to avoid accidental data loss.

**How we execute:** Lifecycle policies run daily and apply to all blobs matching the rule's filters (container, prefix, tags). Use JSON rule definitions for version control and Infrastructure-as-Code (IaC) deployment via Bicep or Terraform.

For dynamic deletion (e.g., user-triggered erasure, legal hold checks), deploy an Azure Function with a timer trigger that queries storage, checks metadata (legal hold flags, exception tags), and calls the Blob Delete API. The function logs each deletion attempt to Azure Monitor for audit trails.

Below is a minimal Python Azure Function that deletes blobs older than a threshold, respecting legal hold flags:

In [ ]:
import os
import logging
from datetime import datetime, timedelta
from azure.storage.blob import BlobServiceClient
from azure.identity import DefaultAzureCredential

def main(mytimer):
    account_url = os.environ["STORAGE_ACCOUNT_URL"]
    container_name = os.environ["CONTAINER_NAME"]
    retention_days = int(os.environ["RETENTION_DAYS"])
    
    credential = DefaultAzureCredential()
    blob_service = BlobServiceClient(account_url, credential=credential)
    container_client = blob_service.get_container_client(container_name)
    
    cutoff = datetime.utcnow() - timedelta(days=retention_days)
    
    for blob in container_client.list_blobs(include=["metadata"]):
        if blob.metadata.get("legal_hold") == "true":
            logging.info(f"Skipping {blob.name} (legal hold)")
            continue
        if blob.last_modified < cutoff:
            container_client.delete_blob(blob.name)
            logging.info(f"Deleted {blob.name}")

This function runs on a schedule (e.g., daily), checks each blob's last-modified date and legal hold metadata, and deletes eligible blobs. Deploy it via Azure Functions Core Tools or CI/CD pipelines, and monitor execution in Azure Monitor.

**Leader Decision Checklist:**
- **What to decide:** Lifecycle rule scope (which containers, prefixes, tags), deletion schedule (daily, weekly), legal hold metadata schema, exception approval workflow.
- **Who owns it:** Data Platform implements lifecycle rules and Functions; Legal/DPO approves exception workflow; Ops monitors deletion success rates.
- **Evidence to collect:** Lifecycle rule JSON definitions, Function deployment logs, legal hold metadata schema, exception approval form template.
- **Success metric:** Lifecycle rules active for all data classes; Function runs without errors; deletion success rate >98%; oldest object age <90 days.

**Log sanitization:**
- For logs containing PII, deploy a sanitization step before extending retention. Use Azure Data Loss Prevention (DLP) services, regex/ML classifiers, or tokenization to detect and redact PII.
- Define acceptance criteria for sanitization (e.g., 99% PII recall, manual spot-check of 1% of logs) and document the process in your retention register.

**Resourcing and budget:**
- Expect 0.5–1.0 FTE engineer for 4 weeks to implement lifecycle rules, Functions, and monitoring; 0.2 FTE Legal/DPO for policy review and sign-off.
- Azure costs: minimal (lifecycle management is free; Functions consumption tier ~$1–5/month; Azure Monitor ~$10–50/month depending on log volume).

**Evidence to collect:** Lifecycle rule JSON, Function source code and deployment logs, sanitization acceptance criteria, deletion success rate dashboard, Ops sign-off after automation dry run.

## Orchestrate workflows and build compliance dashboards

For low-code orchestration, assemble Azure Logic Apps that run weekly: query storage, call OpenAI APIs, check legal hold flags, and post status to Teams or Email. Use parallel branches per data class so a failed step for one class doesn't block others. If you're looking to align your teams and processes for scalable AI initiatives, our [step-by-step roadmap to successful AI agent projects](/article/your-step-by-step-roadmap-to-successful-ai-agent-projects-6) offers actionable guidance.

**How we execute:** Logic Apps provide a visual designer for workflows. Add a Recurrence trigger (weekly), then add actions: List Blobs (Azure Storage connector), Condition (check metadata), Delete Blob, Send Email (Office 365 connector). Deploy via Azure portal or ARM templates.

Publish a dashboard showing deletion success rates, exception counts, oldest-object-age, and ZDR/data zone status. Azure Monitor Workbooks can visualize logs across storage, Functions, and Logic Apps. Schedule a monthly review with stakeholders to address drift promptly. For frameworks and case studies on quantifying the business value of your AI initiatives, explore our article on [measuring the ROI of AI in business](/article/measuring-the-roi-of-ai-in-business-frameworks-and-case-studies-2).

**Leader Decision Checklist:**
- **What to decide:** Workflow triggers (daily, weekly), notification recipients (Legal, Security, Ops), dashboard KPIs (deletion success rate, oldest object age, exception count, storage cost trend), review cadence (monthly).
- **Who owns it:** Data Platform builds workflows and dashboards; Legal/DPO defines KPIs; Ops owns monthly review and drift remediation.
- **Evidence to collect:** Logic App workflow diagrams, dashboard screenshots, monthly review checklist template, KPI definitions and owners.
- **Success metric:** Workflows run on schedule with <5% failure rate; dashboard accessible to stakeholders; monthly review completed with documented action items.

**Value tracking and ROI:**
- Track KPIs: reduced audit prep hours (baseline vs. post-automation), storage cost savings (GB deleted × cost per GB), regulatory risk reduction (audit findings, fines avoided).
- Assign owners to each KPI and report quarterly to leadership. Use these metrics to justify continued investment and scaling.

**Change management and team enablement:**
- Update internal policies to reflect new retention rules; publish user-facing guidance on prompt hygiene (e.g., "Do not paste PII into prompts").
- Deploy DLP banners in GenAI UIs to remind users of data handling rules; require compliance acknowledgements for high-risk use cases.
- Conduct training sessions for App Owners, Data Platform, and end users; provide FAQ and escalation paths for retention questions.
- Deliverables: updated policy documents, training slide deck, DLP banner copy, compliance acknowledgement form, FAQ document.

**Template artifacts:**
- Retention register schema (data class, retention period, lawful basis, owner, last review date).
- Exception/hold approval form (requestor, data class, hold reason, approval date, release criteria).
- Quarterly review checklist (KPIs reviewed, drift identified, action items assigned, sign-off).
- Audit evidence pack index (ZDR screenshots, lifecycle rules, deletion logs, vendor agreements, DPIA, RoPA, training records).

**Policy language samples:**
- Storage limitation clause: "Personal data processed by GenAI systems shall be retained only as long as necessary for the specified purpose, with retention periods defined per data class in the retention register."
- Legal hold clause: "Data subject to legal hold shall be preserved until the hold is released by Legal/DPO, notwithstanding standard retention periods."
- DSR fulfillment clause: "Data subjects may request access or erasure of their personal data; requests shall be fulfilled within 30 days per GDPR Article 15/17, with deletion propagated across all systems and backups."

**Sector-specific conflict resolution:**
- If financial recordkeeping mandates 7-year retention but GDPR storage limitation suggests shorter periods, document the conflict and the chosen retention period with Legal/DPO approval.
- Use a decision flow: identify conflicting rules → consult Legal/DPO → document exception and rationale → update retention register → obtain sign-off.

**Evidence to collect:** Logic App workflow diagrams, Azure Monitor Workbook screenshots, monthly review meeting notes, KPI tracking spreadsheet, policy update diffs, training attendance records, template artifact library, sector-specific conflict resolution log.

## Put retention on rails in 30 days

**Week 1:** Classify data, map retention drivers, run DPIA, update RoPA, execute vendor DPAs, conduct discovery sweep, define DSR workflow, and draft retention register. Legal/DPO and Security sign off on classification and retention windows.

**Week 2:** Enable ZDR, centralize artifacts in governed storage, tag containers, implement RBAC, define backup retention policy, and document compensating controls. Security signs off on ZDR trade-offs and storage topology.

**Week 3:** Deploy lifecycle rules in non-production, build and test Azure Function for dynamic deletion, implement log sanitization, and dry-run automation. Data Platform and Ops validate deletion success rates and oldest object age.

**Week 4:** Promote lifecycle rules and Function to production, deploy Logic Apps for orchestration, publish compliance dashboard, conduct pilot review, and prepare audit evidence pack. Legal/DPO, Security, and Ops sign off on production readiness. Schedule first monthly review.

**Go/no-go gate:** Before enterprise rollout, confirm deletion error rate <2%, audit log completeness 100%, vendor baseline captured, and all sign-offs obtained. If criteria are not met, remediate and re-test before scaling.

Track deletion success rates, exception counts, and oldest-object-age weekly. Review the retention register quarterly with Legal, Security, and business owners to catch new use cases or regulatory changes. Update lifecycle rules and workflows as needed, and maintain audit evidence (screenshots, logs, sign-offs) in a central repository for compliance reviews. Your 30-day roadmap delivers a production-ready retention policy, automated deletion, and audit-ready evidence—giving you confidence that your GenAI systems meet GDPR, HIPAA, and sector-specific mandates while controlling storage costs and regulatory risk.